# Making an SQLite sharable/portable DB out of raw CSV data

## 1) Finding a source

In [1]:
link = 'https://www.kaggle.com/datasets/stereobooster/yale-som?resource=download'

## 2) Converting a CSV file into a python list

In [2]:
import csv

In [3]:
with open('files/yale_data.csv', 'r') as f:
    file_csv = csv.reader(f)
    data = list(file_csv)

## 3) Shaping raw data into a more structured form

In [4]:
data[0]

['name', 'action', 'yaleGrade', 'countryAlpha2', 'country', 'industry']

In [5]:
companies = [{data[0][j] : data[i][j] for j in range(len(data[0]))} for i in range(1, len(data))]

#### Here is a more readable version:

In [6]:
companies_old = []
for i in range(1, len(data)):
    info = {}
    for j in range(len(data[0])):
        info[data[0][j]] = data[i][j]
    companies_old.append(info)

In [7]:
companies == companies_old

True

## 4) Creating a DB for the data & optimising space efficiency

In [8]:
import sqlite3

In [9]:
con = sqlite3.connect('files/yale_data.sqlite')
cur = con.cursor()

In [10]:
data[0]

['name', 'action', 'yaleGrade', 'countryAlpha2', 'country', 'industry']

In [11]:
max_name_len = 0
for i in range(len(data)):
    if max_name_len < len(data[i][0]):
        max_name_len = len(data[i][0])

max_name_len

63

In [12]:
max_action_len = 0
for i in range(len(data)):
    if max_action_len < len(data[i][1]):
        max_action_len = len(data[i][1])

max_action_len

147

In [13]:
max_country_len = 0
for i in range(len(data)):
    if max_country_len < len(data[i][4]):
        max_country_len = len(data[i][4])

max_country_len

20

In [14]:
max_industry_len = 0
for i in range(len(data)):
    if max_industry_len < len(data[i][5]):
        max_industry_len = len(data[i][5])

max_industry_len

22

When creating the table, space optimization comes at the cost of flexibility of newly added data.

If maximum space optimization is needed, the table creation query will be as follows:
```python
query = f'''
CREATE TABLE IF NOT EXISTS companies (
    id INT PRIMARY KEY NOT NULL,
    name VARCHAR({max_name_len}),
    action VARCHAR({max_action_len}),
    yale_grade CHAR(1),
    country_alpha_2 CHAR(2),
    country VARCHAR({max_country_len}),
    industry VARCHAR({max_industry_len})
)'''
```
For maximum flexibility to add new entries, the query changes slightly and looks like this:
```python
query = f'''
CREATE TABLE IF NOT EXISTS companies (
    id INT PRIMARY KEY NOT NULL,
    name TEXT,
    action TEXT,
    yale_grade CHAR(1),
    country_alpha_2 CHAR(2),
    country TEXT,
    industry TEXT
)'''
```
A compromise query between the two also exists, and would look something like this:
```python
query = f'''
CREATE TABLE IF NOT EXISTS companies (
    id INT PRIMARY KEY NOT NULL,
    name VARCHAR({max_name_len + 100}),
    action VARCHAR({max_action_len + 100}),
    yale_grade CHAR(1),
    country_alpha_2 CHAR(2),
    country VARCHAR({max_country_len + 10}),
    industry VARCHAR({max_industry_len + 20})
)'''
```
In this project, I'll go with the first option.

In [15]:
create_query = f'''
CREATE TABLE IF NOT EXISTS Companies (
    id INT PRIMARY KEY NOT NULL,
    name VARCHAR({max_name_len}),
    action VARCHAR({max_action_len}),
    yale_grade CHAR(1),
    country_alpha_2 CHAR(2),
    country VARCHAR({max_country_len}),
    industry VARCHAR({max_industry_len})
)'''

cur.execute(create_query)

In [16]:
cur.execute('SELECT * FROM sqlite_master')
res = cur.fetchall()

res

[('table',
  'Companies',
  'Companies',
  2,
  'CREATE TABLE Companies (\n    id INT PRIMARY KEY NOT NULL,\n    name VARCHAR(63),\n    action VARCHAR(147),\n    yale_grade CHAR(1),\n    country_alpha_2 CHAR(2),\n    country VARCHAR(20),\n    industry VARCHAR(22)\n)'),
 ('index', 'sqlite_autoindex_Companies_1', 'Companies', 3, None)]

## 5) Adding the data to the table

In [17]:
create_query

'\nCREATE TABLE IF NOT EXISTS Companies (\n    id INT PRIMARY KEY NOT NULL,\n    name VARCHAR(63),\n    action VARCHAR(147),\n    yale_grade CHAR(1),\n    country_alpha_2 CHAR(2),\n    country VARCHAR(20),\n    industry VARCHAR(22)\n)'

In [18]:
companies[0]

{'name': 'Acerinox',
 'action': 'still operating in Russia',
 'yaleGrade': 'f',
 'countryAlpha2': 'ES',
 'country': 'Spain',
 'industry': 'Materials'}

In [19]:
for i in range(len(companies)):
    add_query = f'''
    INSERT INTO companies
    VALUES(?,?,?,?,?,?,?)
    '''
    cur.execute(add_query, (i+1,
        companies[i]['name'],
        companies[i]['action'],
        companies[i]['yaleGrade'],
        companies[i]['countryAlpha2'],
        companies[i]['country'],
        companies[i]['industry']
    ))

In [20]:
select_query = '''
SELECT *
FROM companies
'''

cur.execute(select_query)
res = cur.fetchall()

len(res) == len(companies)

True

In [21]:
select_query = '''
SELECT *
FROM companies
WHERE id = 1
'''

cur.execute(select_query)
res = cur.fetchall()

res

[(1, 'Acerinox', 'still operating in Russia', 'f', 'ES', 'Spain', 'Materials')]

## 6) Let's see what's in the DB

In [22]:
# Different companies having the same name:

select_query = '''
SELECT name, COUNT(name)
FROM companies
GROUP BY name
HAVING COUNT(name) > 1
'''

cur.execute(select_query)
res = cur.fetchall()

res

[('JCB', 2), ('Merck', 2), ('Systemair', 2)]

In [23]:
# Number of companies that are very responsible (A), 
# that (might) have a good heart (B, C), 
# and that you should boycott (D, F):

select_query = '''
SELECT yale_grade, COUNT(yale_grade)
FROM companies
GROUP BY yale_grade
'''

cur.execute(select_query)
res = cur.fetchall()

res

[('a', 325), ('b', 498), ('c', 173), ('d', 162), ('f', 230)]

As seen above, by the time of the data report the largest number of listed companies are in the process of leaving Russia's market. Let's see the overall tendency

In [24]:
# Comparing A + B companies to C + D + F ones:

select_query = '''
SELECT COUNT(yale_grade)
FROM companies
GROUP BY yale_grade
'''

cur.execute(select_query)
res = cur.fetchall()

a_b = sum((res[0][0], res[1][0]))
c_d_f = sum((res[2][0], res[3][0], res[4][0]))

a_b > c_d_f

True

Okay, there is hope :)

## That's it. Now we save up and close the connection

In [25]:
con.commit()
con.close()